Investigate correlations between our predictors and labels

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
import pickle
import scipy.stats

In [2]:
file_suffix='mar9'

with open('../OtherData/CombinedDFAdjPopulationExcessMobility_%s.pkl' % (file_suffix), 'rb') as handle:
    df = pickle.load(handle)

In [3]:
#combined2.where(combined2["accumulated_cases"] == 0).dropna()
df.head()

,Date,Region,Imported Cases,Internal Movement,Excess Imported Cases,Excess Internal Movement,Diff Imported Cases,Diff Internal Movement,Prevalence,accumulated_cases,...,New_Cases_7_Adj100k_6dayago,New_Tests_7_Adj100k_6dayago,Positivity_Rate_7_6dayago,Active_Cases_7_Adj100k_7_6dayago,Excess Internal_Movement_7_Adj100k_6dayago,Diff Internal_Movement_7_Adj100k_6dayago,NewCases_7_Adj100k_next7days,NewTests_7_Adj100k_next7days,Positivity_Rate_7_next7days,Active_Cases_7_Adj100k_next7days
0,2020-04-06,11.0,3.921939,212.960175,-3.931265,-226.041407,1.078971,37.191287,0.000445,36.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.636391,8.909480,0.040816,40.092659
1,2020-04-06,43.0,2.662598,54.670330,-3.809694,-53.559707,0.265948,11.647125,0.000565,51.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.224287,8.074320,0.023810,45.305905
2,2020-04-07,11.0,339.113515,202.206366,-306.583999,-203.395876,40.443814,38.810005,0.000423,36.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.636391,8.591284,0.040816,39.456267
3,2020-04-07,43.0,34.100335,52.911012,-29.197440,-48.437749,6.876160,11.665180,0.000534,51.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.224287,9.868613,0.023810,43.063038
4,2020-04-07,71.0,30.927116,799.524759,-26.389180,-201.275965,4.001431,71.185645,0.005787,355.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,122.899535,0.000000,474.041064


In [4]:
with open('../OtherData/0_test_8_regions_mar9.pkl', 'rb') as handle:
    test_regions = pickle.load(handle)

with open('../OtherData/0_val_7_regions_mar9.pkl', 'rb') as handle:
    val_regions = pickle.load(handle)

print("Number of test regions: %d" % (len(test_regions)))
print("Number of val regions: %d" % (len(val_regions)))

Number of test regions: 7
Number of val regions: 7


In [5]:
all_regions = np.concatenate((val_regions, test_regions))

In [6]:
all_regions

array([22., 44., 32., 43., 23., 21., 11., 41., 42., 51., 24., 61., 62.,
       31.])

In [7]:
df = df.where(df["Region"].isin(all_regions)).dropna()

In [8]:
val_regs = df.where(df["Region"].isin(val_regions)).dropna()
test_regs = df.where(df["Region"].isin(test_regions)).dropna()

# Median start date for a region to be reported in our dataset

In [9]:
first_date = df.sort_values('Date').drop_duplicates(subset=['Region'],keep='first')
first_date_val = val_regs.sort_values('Date').drop_duplicates(subset=['Region'],keep='first')
first_date_test = test_regs.sort_values('Date').drop_duplicates(subset=['Region'],keep='first')

In [10]:
all_regions

array([22., 44., 32., 43., 23., 21., 11., 41., 42., 51., 24., 61., 62.,
       31.])

In [11]:
assert len(first_date)==14
assert len(first_date_val)==7
assert len(first_date_test)==7

In [12]:
dates = list(first_date["Date"])
print(dates[len(dates)//2])

2020-04-23 00:00:00


In [13]:
dates = list(first_date_val["Date"])
print(dates[len(dates)//2])

2020-04-23 00:00:00


In [14]:
dates = list(first_date_test["Date"])
print(dates[len(dates)//2])

2020-04-20 00:00:00


# Population

In [15]:
#https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data

def mean_confidence_interval(df, col, confidence=0.95):
    data = df[col]
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    #print("Mean %.3f\t CI (%.3f - %.3f)"% (m, m-h, m+h))
    ci_low = m-h
    ci_high = m+h
    return m, ci_low, ci_high

In [16]:
def append_to_results_df(results, df1, df2, df3, interest):
    m1, ci1, ci1h = mean_confidence_interval(df1, interest, 0.95)
    m2, ci2, ci2h = mean_confidence_interval(df2, interest, 0.95)
    m3, ci3, ci3h = mean_confidence_interval(df3, interest, 0.95)
    results = results.append({'Column': interest,
                              'All Mean': m1,
                              'All CI Low': ci1,
                              'All CI High': ci1h,
                              'Val Mean': m2,
                              'Val CI Low': ci2,
                              'Val CI High': ci2h,
                              'Test Mean': m3,
                              'Test CI Low': ci3,
                              'Test CI High': ci3h}, ignore_index=True)
    return results

In [17]:
results_df = pd.DataFrame(columns=['Column', 'All Mean', 'All CI Low', 'All CI High',
                                   'Val Mean', 'Val CI Low', 'Val CI High',
                                   'Test Mean', 'Test CI Low', 'Test CI High'])

In [18]:
one_time_labels = ["population", "weighted_mean_poverty", 'median age']
for label in one_time_labels:
    results_df = append_to_results_df(results_df, df.iloc[-14:], val_regs.iloc[-7:], test_regs.iloc[-7:], label)

#all_time_labels = ["active_cases", "new_cases", "new_tests", "positivity_rate", "Imported Cases", "Internal Movement"]
#for label in all_time_labels:
#    results_df = append_to_results_df(results_df, df, val_regs, test_regs, label)

In [19]:
results_df

,Column,All Mean,All CI Low,All CI High,Val Mean,Val CI Low,Val CI High,Test Mean,Test CI Low,Test CI High
0,population,590623.928571,386058.064962,795189.792181,445901.571429,126850.696220,764952.446637,735346.285714,436954.800267,1.033738e+06
1,weighted_mean_poverty,3.957473,3.031993,4.882953,3.439811,1.886877,4.992745,4.475135,3.127915,5.822355e+00
2,median age,29.693661,27.724575,31.662748,28.216108,25.388970,31.043246,31.171214,28.137341,3.420509e+01


In [20]:
all_time_labels = ["active_cases_Adj100k", "new_cases_Adj100k", "new_tests_Adj100k",
                   "positivity_rate", "Imported Cases_Adj100k", "Internal Movement_Adj100k"]
for label in all_time_labels:
    results_df = append_to_results_df(results_df, df, val_regs, test_regs, label)

In [21]:
results_df

,Column,All Mean,All CI Low,All CI High,Val Mean,Val CI Low,Val CI High,Test Mean,Test CI Low,Test CI High
0,population,590623.928571,386058.064962,795189.792181,445901.571429,126850.696220,764952.446637,735346.285714,436954.800267,1.033738e+06
1,weighted_mean_poverty,3.957473,3.031993,4.882953,3.439811,1.886877,4.992745,4.475135,3.127915,5.822355e+00
2,median age,29.693661,27.724575,31.662748,28.216108,25.388970,31.043246,31.171214,28.137341,3.420509e+01
3,active_cases_Adj100k,266.393346,257.456129,275.330563,303.090267,287.600896,318.579638,229.539824,220.925778,2.381539e+02
4,new_cases_Adj100k,21.781280,20.870916,22.691644,24.960245,23.381151,26.539340,18.588748,17.705614,1.947188e+01
5,new_tests_Adj100k,366.627357,354.362044,378.892669,362.485903,343.073465,381.898342,370.786483,355.797072,3.857759e+02
6,positivity_rate,0.051574,0.050194,0.052955,0.056387,0.054186,0.058588,0.046741,0.045102,4.838128e-02
7,Imported Cases_Adj100k,79.066958,76.761497,81.372419,102.168994,98.162147,106.175840,55.866335,54.072827,5.765984e+01
8,Internal Movement_Adj100k,210.259251,202.043772,218.474731,178.888363,171.332952,186.443774,241.764012,227.266994,2.562610e+02
